In [2]:
import PyPlot as plt
plt.pygui(true)

# Import necessary packages
using Luna
using LaTeXStrings
using DelimitedFiles
using Luna
import Luna.PhysData: wlfreq
import FFTW
import Luna: Hankel
import NumericalIntegration: integrate, SimpsonEven
using DSP


## Create example output of a simulation

In [3]:
# From luna example radial.jl

gas = :Ar
pres = 1.2

τ = 20e-15
λ0 = 800e-9

w0 = 40e-6
energy = 2e-9
L = 0.6

R = 4e-3
N = 1024

grid = Grid.RealGrid(L, 800e-9, (400e-9, 2000e-9), 0.2e-12)
q = Hankel.QDHT(R, N, dim=2)

energyfun, energyfun_ω = Fields.energyfuncs(grid, q)

densityfun = let dens0=PhysData.density(gas, pres)
    z -> dens0
end

ionpot = PhysData.ionisation_potential(gas)
ionrate = Ionisation.ionrate_fun!_PPTcached(gas, λ0)

responses = (Nonlinear.Kerr_field(PhysData.γ3_gas(gas)),)
             #Nonlinear.PlasmaCumtrapz(grid.to, grid.to, ionrate, ionpot))

linop = LinearOps.make_const_linop(grid, q, PhysData.ref_index_fun(gas, pres))

normfun = NonlinearRHS.const_norm_radial(grid, q, PhysData.ref_index_fun(gas, pres))

inputs = Fields.GaussGaussField(λ0=λ0, τfwhm=τ, energy=energy, w0=w0, propz=-0.3)

Eω, transform, FT = Luna.setup(grid, q, densityfun, normfun, responses, inputs)

# statsfun = Stats.collect_stats(grid, Eω, Stats.ω0(grid))
output = Output.MemoryOutput(0, grid.zmax, 201)
Luna.run(Eω, grid, linop, transform, FT, output)

┌ Info: Freq limits 0.15 - 0.75 PHz
└ @ Luna.Grid C:\Users\muelderk\.julia\packages\Luna\7FAkO\src\Grid.jl:40
┌ Info: Samples needed: 899.38, samples: 1024, δt = 222.38 as
└ @ Luna.Grid C:\Users\muelderk\.julia\packages\Luna\7FAkO\src\Grid.jl:44
┌ Info: Requested time window: 200.0 fs, actual time window: 227.7 fs
└ @ Luna.Grid C:\Users\muelderk\.julia\packages\Luna\7FAkO\src\Grid.jl:46
┌ Info: Grid: samples 512 / 1024, ωmax 7.06e+15 / 1.41e+16
└ @ Luna.Grid C:\Users\muelderk\.julia\packages\Luna\7FAkO\src\Grid.jl:78
┌ Info: Found cached PPT rate for 15.76 eV, 800.0 nm
└ @ Luna.Ionisation C:\Users\muelderk\.julia\packages\Luna\7FAkO\src\Ionisation.jl:145
┌ Info: Found FFTW wisdom at C:\Users\muelderk\.julia\scratchspaces\30eb0fb0-5147-11e9-3356-d75b018717ce\lunacache\FFTWcache_1threads
└ @ Luna.Utils C:\Users\muelderk\.julia\packages\Luna\7FAkO\src\Utils.jl:90
┌ Info: FFTW wisdom saved to C:\Users\muelderk\.julia\scratchspaces\30eb0fb0-5147-11e9-3356-d75b018717ce\lunacache\FFTWcache_1t

In [4]:
zout = output.data["z"]
println(zout[1])
println(zout[end])
println(length(zout))

0.0
0.6
201


In [150]:
ω = grid.ω
t = grid.t

zout = output.data["z"]
Eout = output.data["Eω"]

Fields.prop_taylor!(Eout, grid, [0, 0, 300e-30], λ0)

Erout = (q \ Eout)

257×1024×201 Array{ComplexF64, 3}:
[:, :, 1] =
  -2.34292e-7-4.41662e-8im  …  -1.01554e-23-1.90628e-24im
   1.59846e-8-3.08364e-9im      3.62064e-11+2.12526e-10im
   8.97523e-9-2.77051e-8im      -5.7681e-11+2.34703e-10im
   1.21007e-7-6.28739e-8im      2.16706e-10+1.00798e-10im
    3.8584e-8+9.63368e-8im     -1.26526e-10+1.09877e-10im
  -8.92988e-8-1.18054e-8im  …   4.70205e-10-6.14986e-10im
  -8.45914e-8-2.49019e-8im      -1.13481e-9-5.27438e-10im
  -2.08034e-7-1.01065e-7im     -8.35557e-10-1.87866e-9im
   -8.9144e-8-3.51533e-8im       2.77944e-9+1.76659e-9im
    6.1071e-8+3.98371e-8im      -2.8956e-10-7.76491e-11im
             ⋮              ⋱  
   6.65665e-8+2.34654e-9im     -9.09626e-11+2.82848e-10im
   4.79547e-8+9.02329e-9im     -7.74828e-10+2.40357e-10im
   2.79511e-8-2.1242e-8im   …  -2.10032e-10+5.36035e-12im
 -8.86123e-10-2.83079e-9im      3.50275e-10+3.21114e-10im
   5.28825e-9+2.33026e-8im     -2.72348e-10-3.72146e-10im
   2.89658e-8-3.27925e-9im     -3.58564e-10-1.10732e-

In [18]:
print("Integrating field along r...")
Ẽω = zeros(ComplexF64, (size(Eout, 1), size(Eout, 3)))      # set up new array for Ẽω = Ẽ(ω, z); COMPLEX electric field amplitude in FREQUENCY domain INTEGRATED along r (from 0 to infinity to my knowledge)      
    for i = 1:size(Eout, 1), j = 1:size(Eout, 3)
        Ẽω[i,j] = Hankel.integrateK(Eout[i,:,j], q)             # integrate along r (technically: k)
    end
println("done.")

Integrating field along r...

UndefVarError: UndefVarError: `Eout` not defined

Plot phase how I did before, directly by applying the angle function and then unwrap it.

In [1]:
Ẽω_in = Ẽω[:, end]

Iω_in = abs2.(Ẽω_in)

ϕω_in = -angle.(Ẽω_in)


fig, ax1 = plt.subplots()
    ax1.plot(ω.*1e-15, Iω_in, label="intensity")
    ax2 = ax1.twinx()
    ax2.plot(ω.*1e-15, unwrap(ϕω_in), label="phase", color="orange")
    #ax1.set_xlim(1.8 , 3)
    #ax2.set_ylim(-50, 50)
    ax1.set_xlabel("ω [PHz]")
    ax1.set_ylabel("I (arb. units)")
    ax2.set_ylabel("phase [rad]")
    ax1.legend(loc="upper left")
    ax2.legend(loc="upper right")
    plt.tight_layout()
    plt.gcf()



UndefVarError: UndefVarError: `Ẽω` not defined

Now first eliminate large linear contribution by multiplying the complex electric field with an exponential factor that contains a linear phase.

In [2]:
τ = -length(grid.t)*(grid.t[2] - grid.t[1])/2

Ẽω_in_shifted = Ẽω_in .* exp.(-1im.*grid.ω.*τ) # shift to -t_max before unwrapping
Aω = abs.(Ẽω_in_shifted) # spectral amplitude
ϕω = unwrap(-angle.(Ẽω_in_shifted)) # spectral phase
#Ag = Maths.BSpline(oldgrid.ω, Aω).(newgrid.ω)
ϕg = ϕω .- grid.ω*τ # shift to t=0
#Egm[:, midx] = scale * Ag .* exp.(1im*ϕg)

UndefVarError: UndefVarError: `grid` not defined

In [3]:
fig, ax1 = plt.subplots()
    ax1.plot(ω.*1e-15, Aω, label="intensity")
    ax2 = ax1.twinx()
    ax2.plot(ω.*1e-15, ϕg, label="phase", color="orange")
    #ax1.set_xlim(1.8 , 3)
    #ax2.set_ylim(-50, 50)
    ax1.set_xlabel("ω [PHz]")
    ax1.set_ylabel("I (arb. units)")
    ax2.set_ylabel("phase [rad]")
    ax1.legend(loc="upper left")
    ax2.legend(loc="upper right")
    plt.tight_layout()
    plt.gcf()

UndefVarError: UndefVarError: `plt` not defined

In [4]:
function phase_blanking(x, y, ϕ; level=0.01)

    val = level*maximum(y)
    maxidx = argmax(y)
    xmax = x[maxidx]

    lefti = findlast((x .< xmax) .& (y .< val))
    righti = findfirst((x .> xmax) .& (y .< val))

    ϕ_blanked = copy(ϕ)
    ϕ_blanked[1:lefti-1] .= NaN
    ϕ_blanked[righti+1:end] .= NaN
    ϕ_blanked .-= minimum(filter(!isnan, ϕ_blanked))

    return (ϕ_blanked)
end

phase_blanking (generic function with 1 method)

In [5]:
ϕ_blanked = phase_blanking(ω, Aω, ϕg; level=0.1)


UndefVarError: UndefVarError: `ω` not defined

In [6]:
fig, ax1 = plt.subplots()
    ax1.plot(ω.*1e-15, Aω, label="intensity")
    ax2 = ax1.twinx()
    ax2.plot(ω.*1e-15, ϕ_blanked, label="phase", color="orange")
    #ax1.set_xlim(1.8 , 3)
    #ax2.set_ylim(-50, 50)
    ax1.set_xlabel("ω [PHz]")
    ax1.set_ylabel("I (arb. units)")
    ax2.set_ylabel("phase [rad]")
    ax1.legend(loc="upper left")
    ax2.legend(loc="upper right")
    plt.tight_layout()
    plt.gcf()

UndefVarError: UndefVarError: `plt` not defined

Phase blanking for 2d array

In [7]:
τ = -length(grid.t)*(grid.t[2] - grid.t[1])/2

Ẽω_tot_shifted = Ẽω .* exp.(-1im.*grid.ω.*τ) # shift to -t_max before unwrapping
Aω_tot = abs.(Ẽω_tot_shifted) # spectral amplitude
ϕω_tot = unwrap(-angle.(Ẽω_tot_shifted), dims=1) # spectral phase



UndefVarError: UndefVarError: `grid` not defined

In [8]:
fig, ax1 = plt.subplots()
    ax1.plot(ω.*1e-15, Aω_tot[:, end], label="intensity")
    ax2 = ax1.twinx()
    ax2.plot(ω.*1e-15, ϕω_tot[:, end], label="phase", color="orange")
    #ax1.set_xlim(1.8 , 3)
    #ax2.set_ylim(-50, 50)
    ax1.set_xlabel("ω [PHz]")
    ax1.set_ylabel("I (arb. units)")
    ax2.set_ylabel("phase [rad]")
    ax1.legend(loc="upper left")
    ax2.legend(loc="upper right")
    plt.tight_layout()
    plt.gcf()

UndefVarError: UndefVarError: `plt` not defined

In [9]:
function blank_phase(x, I, ϕ; level=0.1)
    
    # works for 1D and 2D I, ϕ arrays
    
    if size(I) != size(ϕ)
        error("Error: I and ϕ have to be of the same size.")
    end
    
    num_col = size(I, 2)  # Number of columns in I and ϕ
    ϕ_blanked = similar(ϕ)  # Allocate array to store results

    # Iterate over each column in I
    for i in 1:num_col
        I_col = I[:, i]
        ϕ_col = ϕ[:, i]

        val = level * maximum(I_col)    # threshold intensity
        maxidx = argmax(I_col)
        xmax = x[maxidx]                # frequency/time of threshold intensity

        # find left and right indices between which intensity is above
        # lefti = findlast((x .< xmax) .& (I_col .< val))
        # righti = findfirst((x .> xmax) .& (I_col .< val))

        lefti = findfirst((x .< xmax) .& (I_col .> val))
        righti = findlast((x .> xmax) .& (I_col .> val))

        ϕ_blanked_col = copy(ϕ_col)
        
        if !isnothing(lefti)
            ϕ_blanked_col[1:lefti-1] .= NaN
        end
        if !isnothing(righti)
            ϕ_blanked_col[righti+1:end] .= NaN
        end

        # shift phase values to zero (convinient for comparing the phase graphs)
        ϕ_blanked_col .-= minimum(filter(!isnan, ϕ_blanked_col))

        ϕ_blanked[:, i] = ϕ_blanked_col
    end
    
    return ϕ_blanked
end

blank_phase (generic function with 1 method)

In [10]:
ϕ_tot_blanked = blank_phase(ω, Aω_tot[:,1], ϕω_tot[:,1])#; level=0.1)

println(ϕ_tot_blanked[:,end])

UndefVarError: UndefVarError: `ω` not defined

In [11]:
fig, ax1 = plt.subplots()
    ax1.plot(ω.*1e-15, Aω_tot[:,1], label="intensity", marker="o")
    ax1.axhline(0.1*maximum(Aω_tot[:,1]), color="grey", linestyle="dashed")
    ax2 = ax1.twinx()
    ax2.plot(ω.*1e-15, ϕ_tot_blanked[:,1], label="phase", color="orange")
    #ax1.set_xlim(1.8 , 3)
    #ax2.set_ylim(-50, 50)
    ax1.set_xlabel("ω [PHz]")
    ax1.set_ylabel("I (arb. units)")
    ax2.set_ylabel("phase [rad]")
    ax1.legend(loc="upper left")
    ax2.legend(loc="upper right")
    plt.tight_layout()
    plt.gcf()
    # plt.show()

UndefVarError: UndefVarError: `plt` not defined

In [13]:
τ = -length(grid.t)*(grid.t[2] - grid.t[1])/2

Erout_shifted = Erout .* exp.(-1im.*grid.ω.*τ) # shift to -t_max before unwrapping
Aωr = abs.(Erout_shifted) # spectral amplitude
ϕωr = unwrap(-angle.(Erout_shifted), dims=1) # spectral phase

UndefVarError: UndefVarError: `grid` not defined

In [14]:
ϕωr_blanked = phase_blanking2D(ω, Aωr, ϕωr; level=0.1)


UndefVarError: UndefVarError: `phase_blanking2D` not defined